In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

load_dotenv()

True

In [2]:
model=ChatGroq(model='llama-3.1-70b-versatile')

In [3]:
print("-----Prompt from Template------")
template="Tell me a joke {topic}."
prompt_template=ChatPromptTemplate.from_template(template)

prompt=prompt_template.invoke({"topic":"cats"})
result=model.invoke(prompt)
print(result.content)

-----Prompt from Template------
Why did the cat go to the vet?

Because it was feeling a little cat-atrophic.

Or here's another one:

Why did the cat climb up the tree?

To purr-suade the birds to come down.

Or this one:

What did the cat say when it was happy?

"I'm feline great!"


In [4]:
# Part 2: Prompt with Multiple Placeholders

print("\n ----- Prompt with multiple placeholders ----\n")
template_multiple="""You are a helpful assistant.
Human: Tell me a {adjective} short story about a {animal}.
Assistant:
"""

prompt_multiple=ChatPromptTemplate.from_template(template_multiple)
prompt=prompt_multiple.invoke({"adjective":"funny","animal":"panda"})

result=model.invoke(prompt)


 ----- Prompt with multiple placeholders ----



In [9]:
result.content

"One day, in a bamboo forest in China, there was a panda named Ping. Ping was a little clumsy and loved to dance. One sunny afternoon, while his panda friends were busy munching on bamboo shoots, Ping decided to practice his best ballet moves.\n\nAs he twirled and leaped through the forest, his big furry feet stomped on a beehive hidden in the underbrush. The bees, startled by the sudden noise, flew out and started chasing Ping.\n\nPanicked, Ping began to dance even faster, waving his paws wildly in the air. The bees, confused by the panda's crazy dance moves, started to get dizzy and disoriented.\n\nEventually, the bees got so tired from chasing Ping that they flew back to their hive, exhausted. Ping, still dancing, tripped and fell into a nearby mud pit.\n\nCovered in mud, Ping looked up at his friends, who were now laughing hysterically at the sight of the muddy panda. From that day on, Ping was known as the greatest (and most clumsy) panda dancer in the forest."

In [8]:
from pprint import pprint
pprint(result)

AIMessage(content="One day, in a bamboo forest in China, there was a panda named Ping. Ping was a little clumsy and loved to dance. One sunny afternoon, while his panda friends were busy munching on bamboo shoots, Ping decided to practice his best ballet moves.\n\nAs he twirled and leaped through the forest, his big furry feet stomped on a beehive hidden in the underbrush. The bees, startled by the sudden noise, flew out and started chasing Ping.\n\nPanicked, Ping began to dance even faster, waving his paws wildly in the air. The bees, confused by the panda's crazy dance moves, started to get dizzy and disoriented.\n\nEventually, the bees got so tired from chasing Ping that they flew back to their hive, exhausted. Ping, still dancing, tripped and fell into a nearby mud pit.\n\nCovered in mud, Ping looked up at his friends, who were now laughing hysterically at the sight of the muddy panda. From that day on, Ping was known as the greatest (and most clumsy) panda dancer in the forest.", 

In [10]:
# Part 3: Prompt with System and Human messages (using Tuples)
messages = [
    ('system', 'You are a comedian who tells jokes about {topic}.'),
    ('human', 'Tell me {joke_count} jokes.')
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({'topic': 'lawyers', 'joke_count': 4})
print(f"----- Prompt with Sytem and Human messages (Tuple) -----")
print(prompt)

----- Prompt with Sytem and Human messages (Tuple) -----
messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 4 jokes.', additional_kwargs={}, response_metadata={})]


In [11]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [12]:
# This work
messages = [
    ('system', 'You are a comedian who tells jokes about {topic}.'),
    HumanMessage(content="Tell me 3 jokes")
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({'topic': 'lawyers'})

# # This does not work
# messages = [
#     ('system', 'You are a comedian who tells jokes about {topic}.'),
#     HumanMessage(content="Tell me {joke_count} jokes")
# ]
# prompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = prompt_template.invoke({'topic': 'lawyers', 'joke_count': 4})

print(f"Prompt with System and Human Messages:")
print(prompt)

Prompt with System and Human Messages:
messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes', additional_kwargs={}, response_metadata={})]


# Chains

In [14]:
# Langchain Expression Language (LCEL)  => chain= prompt | model
# result=chain.invoke({"key":"value"})  this key is passed to prompt which then passes its output to model.

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a comedian who tells jokes about {topic}'),
        ('human'), 'Tell me {joke_count} jokes.'
    ]
)

# Combined chain using Langchain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()
result = chain.invoke({'topic': 'lawyers', 'joke_count': 3})
print(result)

Here are three lawyer jokes for you:

1. Why did the lawyer's client bring a ladder to the courtroom? Because he wanted to take his case to a higher court. 

2. Why did the lawyer's dog go to the vet? Because it was feeling a little ruff, but the lawyer just wanted to get a paws-itive verdict.

3. Why did the lawyer cross the road? To get to the other side... of the argument, where the billable hours were better.


# Chains - Under the hood

In [15]:
# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x)) # Replace the input_variables with the actual values 
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

# Create the RunnableSequence (Equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model],last=parse_output)
response=chain.invoke({'topic': 'lawyers', 'joke_count': 3})
print(response)

A human, that's a great warm-up. Alright, here are three lawyer jokes for you:

1. Why did the lawyer's dog go to the vet?
Because it was feeling ruff justice.

2. Why did the lawyer's client bring a ladder to the courtroom?
Because he wanted to take his case to a higher court.

3. Why did the lawyer cross the road?
To get to the other side... of the argument. (ba-dum-tss)


# Chains Extended

In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a comedian who tells jokes about {topic}'),
        ('human'), 'Tell me {joke_count} jokes.'
    ]
)

uppercase_output=RunnableLambda(lambda x: x.upper())
count_words=RunnableLambda(lambda x: f"word count: {len(x.split())}\n{x}")

chain=prompt_template | model | StrOutputParser() | uppercase_output | count_words

result=chain.invoke({'topic': 'lawyers', 'joke_count': 3})
print(result)

word count: 98
HERE ARE THREE JOKES ABOUT LAWYERS:

1. WHY DID THE LAWYER'S CLIENT BRING A LADDER TO THE COURTROOM? BECAUSE HE WANTED TO TAKE HIS CASE TO A HIGHER COURT.

2. WHY DID THE LAWYER'S DOG GO TO THE VET? IT WAS FEELING A LITTLE RUFF, AND THE LAWYER SAID, 'DON'T WORRY, I'LL PAWS FOR A MOMENT AND SUE THE VET IF THEY DON'T GIVE YOU THE BEST CARE.'

3. WHY DID THE LAWYER CROSS THE ROAD? TO GET TO THE OTHER SIDE... OF THE ARGUMENT, WHERE HE COULD BILL HIS CLIENT FOR THE EXTRA HOUR OF WALKING TIME.


# Parallel Chains

In [17]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnableParallel

In [30]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human","List the main features of the product {product_name}."),
    ]
)

In [21]:
def analyze_pros(features):
    pros_template=ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)

In [22]:
def analyze_cons(features):
    cons_template=ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)

In [23]:
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [24]:
pros_branch=(
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch=(
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [32]:
chain=(
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros":pros_branch, "cons": cons_branch})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [34]:
print(chain.invoke({"product_name": "Pixel 6a"}))

Pros:
Based on the features of the Google Pixel 6a, here are the pros:

1. **Vibrant Display**: The 6.1-inch OLED display provides vibrant colors and a smooth viewing experience, making it perfect for watching videos and browsing the web.
2. **Fast Performance**: The Google Tensor processor offers fast performance, allowing users to multitask and switch between apps seamlessly.
3. **Impressive Camera Capabilities**: The dual-camera setup with a 12.2MP primary sensor and a 12.2MP ultra-wide-angle lens provides excellent camera performance, and the 8MP front camera is great for selfies.
4. **All-Day Battery Life**: The 4410mAh battery provides all-day battery life, and fast charging up to 18W ensures that users can quickly top up their battery when needed.
5. **Timely Software Updates**: The Pixel 6a receives timely software updates directly from Google, ensuring that users have the latest security patches and features.
6. **Secure Biometric Authentication**: The rear-mounted fingerprint

In [35]:
chain=(prompt_template
    | model
    | StrOutputParser())
    
print(chain.invoke({"product_name": "Pixel 6a"}))

The Google Pixel 6a is a mid-range Android smartphone released in 2022. Here are its main features:

1. **Display**: 6.1-inch OLED display with a resolution of 1080 x 2400 pixels, 60Hz refresh rate, and HDR support.

2. **Processor**: Google Tensor chip, a 2.8 GHz octa-core processor paired with 6GB of RAM and 128GB of internal storage.

3. **Camera**: Dual-camera setup with a 12.2 MP primary sensor, 8 MP front camera, and support for 4K video recording at 30fps.

4. **Battery**: 4410mAh battery with 18W fast charging support and wireless charging capabilities.

5. **Software**: The device runs on Android 12 out of the box, with a promise of up to 5 years of security updates.

6. **Biometric security**: In-display fingerprint sensor for secure biometric authentication.

7. **Connectivity**: 5G connectivity, Wi-Fi 6, Bluetooth 5.2, NFC, and a USB-C port.

8. **Durability**: IP67 rating for dust and water resistance, with a durable design featuring a Gorilla Glass 3 front panel and a pol

# Branching Chains

In [36]:
from langchain.schema.runnable import RunnableBranch

In [37]:
positive_feedback_template=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
        "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

In [38]:
negative_feedback_template=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
        "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

In [39]:
neutral_feedback_template=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
        "Generate a request for more details for this neutral feedback: {feedback}."),
    ]
)

In [40]:
escalate_feedback_template=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
        "Generate a message to escalate this feedback to a human agent: {feedback}."),
    ]
)

In [41]:
classification_template=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        ("human",
        "Classify the sentiment of this feedback as positive, negative, neutral or escalate: {feedback}."),
    ]
)

In [42]:
branches=RunnableBranch(
    (
        lambda x:"positive" in x,
        positive_feedback_template | model |StrOutputParser()
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser()
)

In [43]:
classification_chain=classification_template | model | StrOutputParser()

chain= classification_chain | branches

In [44]:
review=" The product is terrible. It broke after just one use and I hate the colour."
result=chain.invoke({"feedback": review})

print(result)

Dear [Customer],

Thank you for taking the time to share your feedback about our product. We apologize that it did not meet your expectations. We are truly sorry to hear that the product broke after just one use, and we understand how frustrating that must be.

We take all issues with our products seriously and would like to make things right. We would like to offer you a replacement or a full refund, whichever you prefer. Please let us know which option you would like to choose, and we will expedite the process as quickly as possible.

Regarding the color, we appreciate your feedback and will take it into consideration for future product development. We understand that color preferences are subjective, and we will do our best to provide a wider range of options in the future.

If you have any further concerns or issues, please do not hesitate to contact us. We value your feedback and would like the opportunity to make things right. Your satisfaction is our top priority, and we appreci

In [45]:
review=" The product is Awesome!!!. I loved everything about it and it's so cool."
result=chain.invoke({"feedback": review})

print(result)

Here's a possible thank-you note in response to the positive feedback:

Dear [Name],

I just wanted to take a moment to express my heartfelt thanks for your wonderful feedback! I'm beyond thrilled to hear that you loved everything about [product/service/experience] and found it to be "awesome" and "so cool." Your kind words mean the world to me, and I'm grateful for your enthusiasm and support.

Thank you again for taking the time to share your thoughts and for being an amazing [customer/user/etc.]. I'm honored to have had the opportunity to serve you and look forward to continuing to provide you with exceptional experiences in the future.

Best regards,
[Your Name]


In [47]:
review=" The product an alright product. Does the job so there's that."
result=chain.invoke({"feedback": review})

print(result)

Here's a possible thank-you note for neutral feedback:

Dear [Customer],

Thank you for taking the time to share your thoughts about our product. We appreciate your feedback and are glad to hear that it's serving its purpose for you. We're always working to improve and appreciate your input in helping us achieve that goal.

Thank you again for your purchase and for being a valued customer.

Best regards,
[Your Name]

This response acknowledges the customer's feedback without taking it as overly positive or negative. It shows appreciation for their input and expresses a commitment to continuous improvement.
